In [2]:
!git clone https://github.com/sahal-mulki/geo-llm

Cloning into 'geo-llm'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 49 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), 1.57 MiB | 3.34 MiB/s, done.


In [3]:
import pandas as pd
import json

ground_truth_yfcc, ground_truth_im = pd.read_csv("/kaggle/input/llms-for-image-geolocation-benchmarks/archive(2)/yfcc4k/yfcc4k/yfcc4k.csv"), pd.read_csv("/kaggle/input/llms-for-image-geolocation-benchmarks/archive(2)/im2gps3k_places365/im2gps3k_places365.csv")

In [4]:
geoclip_preds_yfcc, geoclip_preds_im = pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_yfcc4k.csv"), pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_im2gps3k.csv")

In [5]:
gpt4o_hinted_preds = pd.read_csv("/kaggle/input/gpt-4o-mini-responses-for-geolocation/gpt-4o-mini-geocliphints-addresses-with-coords.csv")

In [81]:
import geopy.distance

In [77]:
dict_results = []

for index, row in gpt4o_hinted_preds.iterrows():
    if type(row["coord_locations"]) == float:
        pass
    else:
        boilerplate = {
            "image_name": None,
            "dataset": None,
            "gpt_4o_coords": None,
            "gpt_4o_address": None,
            "gpt_4o_explanation": None,
            "geoclip_coords": None,
            "ground_truth": None
        }
        
        boilerplate["gpt_4o_coords"] = list((float(row["coord_locations"].split(", ")[0]), float(row["coord_locations"].split(", ")[1])))
        
        boilerplate["dataset"] = row["dataset"]
        
        boilerplate["gpt_4o_explanation"] = row["response"]
        
        boilerplate["image_name"] = row["image name"]
        
        boilerplate["gpt_4o_address"] = json.loads(row["json_locations"])["final_location"]
        
        if row["dataset"] == "yfcc":
            boilerplate["geoclip_coords"] = [geoclip_preds_yfcc.iloc[geoclip_preds_yfcc.index[geoclip_preds_yfcc["Image"] == boilerplate["image_name"]]]["Prediction 1 Latitude"].values[0], geoclip_preds_yfcc.iloc[geoclip_preds_yfcc.index[geoclip_preds_yfcc["Image"] == boilerplate["image_name"]]]["Prediction 1 Longitude"].values[0]]
            boilerplate["ground_truth"] = [ground_truth_yfcc.iloc[ground_truth_yfcc.index[ground_truth_yfcc["Image"] == boilerplate["image_name"]]]["Lat"].values[0], ground_truth_yfcc.iloc[ground_truth_yfcc.index[ground_truth_yfcc["Image"] == "3411014710.jpg"]]["Lon"].values[0]]
        else:
            boilerplate["geoclip_coords"] = [geoclip_preds_im.iloc[geoclip_preds_im.index[geoclip_preds_im["Image"] == boilerplate["image_name"]]]["Prediction 1 Latitude"].values[0], geoclip_preds_im.iloc[geoclip_preds_im.index[geoclip_preds_im["Image"] == boilerplate["image_name"]]]["Prediction 1 Longitude"].values[0]]
            boilerplate["ground_truth"] = [ground_truth_im.iloc[ground_truth_im.index[ground_truth_im["IMG_ID"] == boilerplate["image_name"]]]["LAT"].values[0], ground_truth_im.iloc[ground_truth_im.index[ground_truth_im["IMG_ID"] == boilerplate["image_name"]]]["LON"].values[0]]
        
        import geopy.distance
        dict_results.append(boilerplate)
        

In [80]:
dict_results[523]

{'image_name': '247024863_42eec50d76_91_82927779@N00.jpg',
 'dataset': 'im2gps',
 'gpt_4o_coords': [34.2584462, 108.9361101],
 'gpt_4o_address': "Beilin District, Xi'an, Shaanxi, China",
 'gpt_4o_explanation': "Based on the characteristics of the image, it looks like a street scene featuring cycles with decorative rickshaws, which is common in many parts of Asia. The presence of dense greenery along the street suggests a location with a mild climate.\n\nConsidering the options provided and the style visible in the image, I would guess that this scene is from **Beilin District, Xi'an, Shaanxi, China**. This area is known for its mix of modern and traditional elements, and is a place where rickshaws could commonly be found.",
 'geoclip_coords': [34.48844528198242, 108.720703125],
 'ground_truth': [39.93935, 116.391841]}

In [ ]:

for dict_result in dict_results:
    